In [17]:
import pandas as pd
from IPython.display import display
import glob
from random import shuffle
pd.set_option("display.max_rows", 300)

# 請輸入要計算的檔案

In [2]:
# get the file path in asigned document
file_list = glob.glob(r'score_card\*.xlsx')
file_list

['score_card\\成績表 - 複製.xlsx', 'score_card\\成績表.xlsx']

In [22]:
df = pd.read_excel(file_list[0])
# extract standard score
std_halls = df.iloc[[0],:].reset_index(drop = True)
# extract each player's score
score_table = df.drop([0]).reset_index(drop = True)

# Function which is Need

In [4]:
def cau_total_score(data):
    data = data.set_index('player')
    data['總桿'] = data.sum(axis = 1)
    data['總桿排名'] = data['總桿'].rank(method = 'dense').astype(int)
    data = data.reset_index()
    return data

In [5]:
def random_Select_6halls():
    hall = [e for e in range(1,19)]
    selected_list = []
    
    shuffle(hall)
    
    i = 0
    while i < 7:
        selected_list.append(hall.pop())
        i = i+1
    
    selected_list = sorted(selected_list)
    
    return list(selected_list)

In [6]:
def cau_nnp(data):
    
    data = data.set_index('player')
    
    drop_halls = random_Select_6halls()
    
    print("drop_halls is {}".format(drop_halls))
    
    data=data.drop(columns=drop_halls)
    
    data['sumIn12Halls'] = data.sum(axis = 1)
    
    data['差點'] = (data['sumIn12Halls']*1.5 - 72)*0.8
    
    data = data.reset_index().loc[:,['player','差點']]
    
    return drop_halls,data

In [26]:
def final_table(score_table,nnp_data):
    score_table = score_table.merge(nnp_data, left_on='player',right_on='player',how='left')
    score_table['淨桿'] = score_table['總桿']-score_table['差點']
    score_table['淨桿排名']=score_table['淨桿'].rank(method = 'dense').astype(int)
    score_table = score_table.sort_values(by = ['淨桿排名']).reset_index(drop=True)
    print("被選中的球洞:",drop_hall_list)
    return score_table

In [8]:
def adjusted_table(data, std_df, std_score, adjusted_num):
    
    #transform df into list
    def std_halls_transformer(std_df):
        hall_num =std_df.columns.tolist()
        std_score = std_df.values.tolist()[0] 

        std_list = []
        for i in range(0,19):
            std_list.append([hall_num[i],std_score[i]])
        
        return std_list
    
    
    #find out hall_list if it is match std_score
    def find_hall_number(std_list, std_score):
        tmp = []
        for e in std_list:
            if e[1] == std_score:
                tmp.append(e[0])
        return tmp
    
    #data processing
    std_list = std_halls_transformer(std_df)
    hall_list = find_hall_number(std_list,std_score)
    
    # find each player's score and adjust it to fit nnp234 rule
    tmp = data.loc[:,hall_list].values.tolist()
    #print(tmp)

    adjusted_table = []
    for i in tmp:
        row_element = []    
        for e in i:

            if e >= adjusted_num:
                row_element.append(adjusted_num)
            else:
                row_element.append(e)
    
        adjusted_table.append(row_element)

    adjusted_df = pd.DataFrame(adjusted_table,columns = hall_list)
    
    return adjusted_df

# step1:計算出總桿以及總桿排名

In [23]:
score_table = cau_total_score(score_table)
pd.set_option('display.max_columns', None)
display(score_table)

,player,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,總桿,總桿排名
0,player 1,4,6,3,6,3,6,5,6,5,6,4,7,6,6,4,6,6,5,94,3
1,player 2,5,6,4,7,3,5,5,5,5,4,3,7,5,5,3,5,5,6,88,1
2,player 3,5,5,4,6,3,5,6,6,6,5,4,6,6,6,4,6,6,5,94,3
3,player 4,4,5,4,6,4,5,5,6,5,6,4,6,6,6,3,6,5,5,91,2
4,player 5,5,6,5,7,4,4,5,5,6,6,5,6,5,5,4,5,6,6,95,4
5,player 6,5,6,3,8,4,6,4,7,5,6,5,5,4,5,4,6,6,6,95,4
6,player 7,5,6,3,8,4,6,4,7,5,6,5,5,4,5,4,6,6,6,95,4
7,player 8,5,5,5,6,4,6,5,6,5,7,5,7,4,5,4,5,5,5,94,3
8,player 9,6,5,4,7,3,5,5,7,7,6,4,7,6,7,4,7,5,5,100,8
9,player 10,6,6,5,5,4,5,6,7,6,6,5,6,6,6,4,6,5,5,99,7


# step2: 依據新新西貝里法則調整桿數以及計算每位球員的差點

In [10]:
def score_adjust_BaseOn234rule(score_df, std_df):
    
    new_hall_3 = adjusted_table(score_df, std_df, 3, 5)
    print("3桿洞完成\n")
    new_hall_4 = adjusted_table(score_df, std_df, 4, 7)
    print("4桿洞完成\n")
    new_hall_5 = adjusted_table(score_df, std_df, 5, 9)
    print("5桿洞完成\n")
    player = score_table['player']
    
    new_table = pd.concat([player, new_hall_3, new_hall_4, new_hall_5], axis=1)
    return new_table

In [11]:
table_after_adjust = score_adjust_BaseOn234rule(score_table, std_halls)

3桿洞完成

4桿洞完成

5桿洞完成



In [12]:
drop_hall_list, nnp_data = cau_nnp(table_after_adjust)

drop_halls is [1, 2, 5, 7, 9, 13, 14]


# step3:產出最後的報表(含總桿與淨桿排名)  

## 註: 執行前須確認 step1 與 step2皆完成

In [27]:
output = final_table(score_table,nnp_data)
pd.set_option('display.max_columns', None)
output

被選中的球洞: [1, 2, 5, 7, 9, 13, 14]


,player,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,總桿,總桿排名,差點,淨桿,淨桿排名
0,player 6,5,6,3,8,4,6,4,7,5,6,5,5,4,5,4,6,6,6,95,4,16.8,78.2,1
1,player 7,5,6,3,8,4,6,4,7,5,6,5,5,4,5,4,6,6,6,95,4,16.8,78.2,1
2,player 8,5,5,5,6,4,6,5,6,5,7,5,7,4,5,4,5,5,5,94,3,15.6,78.4,2
3,player 11,4,6,5,7,4,6,7,7,4,7,3,6,4,6,5,7,5,5,98,6,18.0,80.0,3
4,player 14,4,7,6,7,5,7,4,6,4,7,5,8,7,4,4,6,5,5,101,9,20.4,80.6,4
5,player 1,4,6,3,6,3,6,5,6,5,6,4,7,6,6,4,6,6,5,94,3,13.2,80.8,5
6,player 2,5,6,4,7,3,5,5,5,5,4,3,7,5,5,3,5,5,6,88,1,7.2,80.8,5
7,player 4,4,5,4,6,4,5,5,6,5,6,4,6,6,6,3,6,5,5,91,2,9.6,81.4,6
8,player 12,5,5,4,7,4,7,5,6,6,6,6,6,5,5,5,6,5,4,97,5,15.6,81.4,6
9,player 5,5,6,5,7,4,4,5,5,6,6,5,6,5,5,4,5,6,6,95,4,13.2,81.8,7
